<a href="https://colab.research.google.com/github/volinhbao/CS114.K21.KHTN/blob/master/Sarcasm_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center><font color=blue>Sarcasm Detection</font></center></h1>

**Học trò**

1.   Võ Linh Bảo - 18520503
2.   Nguyễn Văn Tiến - 18521489


**Lớp**: CS114.K21.KHTN2018


In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/School_UIT/Semester IV/[CS114] Machine Learning/Working/CS114.K21.KHTN/CaseStudy/SarcsamDetection/

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/School_UIT/Semester IV/[CS114] Machine Learning/Working/CS114.K21.KHTN/CaseStudy/SarcsamDetection


In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

## 1. Mô tả bài toán

- Link tham khảo bài toán: [News Headlines Dataset For Sarcasm Detection
](https://www.kaggle.com/rmisra/news-headlines-dataset-for-sarcasm-detection)

- Nội dung bài toán: Phát hiện trong một câu văn/đoạn văn có tính chất châm biếm hay không (bài toán phân lớp nhị phân).

- Input: một câu văn/đoạn văn bằng tiếng anh.

- Output: Giá trị 0 (non-sarcasm) hoặc 1 (sarcasm) thể hiện câu văn/đoạn văn đầu vào có tính chất châm biếm hay không.

## 2. Quan sát dữ liệu

In [ ]:
DATA_PATH ='dataset/'

In [ ]:
for dirname, _, filenames in os.walk(DATA_PATH):
    for filename in filenames:
        print(os.path.join(dirname, filename))

dataset/Sarcasm_Headlines_Dataset_v2.json
dataset/Sarcasm_Headlines_Dataset.json
dataset/headlines2.txt
dataset/test_data.csv
dataset/crawl/sarcastic/huffpost_new_headlines.csv
dataset/crawl/sarcastic/entertainment_headline_huffpost.csv
dataset/crawl/sarcastic/personal_headline_huffpost.csv
dataset/crawl/sarcastic/shopping_headline_huffpost.csv
dataset/crawl/sarcastic/huffpost_headlines.csv.xlsx
dataset/crawl/non-sarcastic/theonion_headlines.csv
dataset/crawl/videos/huffpost_headline_scraper.mp4.mp4
dataset/crawl/videos/merge_data_huffpost.mp4.mp4
dataset/crawl/videos/theonion_headlines.mp4.mp4


In [ ]:
file1 = pd.read_json('dataset/Sarcasm_Headlines_Dataset.json', lines=True)
file2 = pd.read_json('dataset/Sarcasm_Headlines_Dataset_v2.json', lines=True)

In [ ]:
# Nối hai tập dữ liệu
df = pd.concat([file1,file2],axis=0,sort=False)

# Xem thông tin 5 sample đầu tiên trong tập dữ liệu
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [ ]:
# Bỏ thông tin article_link
df = df.drop('article_link', axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55328 entries, 0 to 28618
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   headline      55328 non-null  object
 1   is_sarcastic  55328 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.3+ MB


Một vài nhận xét:


*   Tập dữ liệu có tổng cộng 55328 mẫu dữ liệu, mỗi mẫu dữ liệu là một câu văn tiếng anh.



## 3. Rút trích đặt trưng

In [ ]:
X = df['headline']
y = df['is_sarcastic']

*Bài viết tham khảo* : [How to Develop a Deep Learning Bag-of-Words Model for Sentiment Analysis (Text Classification)](https://machinelearningmastery.com/deep-learning-bag-of-words-model-sentiment-analysis/)

Sử dụng phương pháp BoW để rút trích đặt trưng.

- Định nghĩa danh sách các `stopwords` (từ không mang giá trị thông tin trong câu).

In [ ]:
stopwords = {'ourselves', 'hers', 'between', 'yourself', 'but',
             'again', 'there', 'about', 'once', 'during', 'out',
             'very', 'having', 'with', 'they', 'own', 'an', 'be',
             'some', 'for', 'do', 'its', 'yours', 'such', 'into',
             'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am',
             'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves',
             'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through',
             'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down',
             'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours',
             'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same',
             'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then',
             'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now',
             'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself',
             'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my',
             'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than'}

In [ ]:
# Sử dụng thư viện sklearn lấy thông tin ma trận token từ tập dữ liệu X
cv = CountVectorizer()

X = cv.fit_transform(X)

Phân chia tập dữ liệu theo tỉ lệ `80%` dữ liệu huấn luyện và `20%` dữ liệu kiểm thử.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

## 5. Chọn mô hình và huấn luyện thuật toán

In [ ]:
Logistic_model = LogisticRegression()
KNN_model = KNeighborsClassifier(n_neighbors=3)
RandomForest_model = RandomForestClassifier()
Bayes_model = GaussianNB()
SVM_model = make_pipeline(StandardScaler(), SVC(gamma='auto'))

### Sử dụng mô hình Logistic

In [ ]:
Logistic_model.fit(X_train, y_train)
print("- The accuracy of logistic model on training set is {}".format(Logistic_model.score(X_train, y_train)))
logistic_pred = Logistic_model.predict(X_test)
print("- The accuracy of logistic model on test set is {}".format(accuracy_score(y_true=y_test, y_pred=logistic_pred)))
print("- Confusion matrix: \n{}".format(classification_report(y_true=y_test, y_pred=logistic_pred)))

- The accuracy of logistic model on training set is 0.9690479418010934
- The accuracy of logistic model on test set is 0.9225555756370866
- Confusion matrix: 
              precision    recall  f1-score   support

           0       0.92      0.94      0.93      5878
           1       0.93      0.90      0.92      5188

    accuracy                           0.92     11066
   macro avg       0.92      0.92      0.92     11066
weighted avg       0.92      0.92      0.92     11066



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


### Sử dụng mô hình k-nestest neighbords (kNN)

In [ ]:
KNN_model.fit(X_train, y_train)
print("- The accuracy of kNN model on training set is {}".format(KNN_model.score(X_train, y_train)))
knn_pred = KNN_model.predict(X_test)
print("- The accuracy of kNN model on test set is {}".format(accuracy_score(y_true=y_test, y_pred=knn_pred)))
print("- Confusion matrix: \n{}".format(classification_report(y_true=y_test, y_pred=knn_pred)))

- The accuracy of kNN model on training set is 0.9302787944512223
- The accuracy of kNN model on test set is 0.7045002711006687
- Confusion matrix: 
              precision    recall  f1-score   support

           0       0.66      0.90      0.76      5878
           1       0.80      0.49      0.61      5188

    accuracy                           0.70     11066
   macro avg       0.73      0.69      0.69     11066
weighted avg       0.73      0.70      0.69     11066



### Mô hình Random Forest

In [ ]:
RandomForest_model.fit(X_train, y_train)
print("- The accuracy of random forest model on training set is {}".format(RandomForest_model.score(X_train, y_train)))
radforest_pred = RandomForest_model.predict(X_test)
print("- The accuracy of random forest model on test set is {}".format(accuracy_score(y_true=y_test, y_pred=radforest_pred)))
print("- Confusion matrix: \n{}".format(classification_report(y_true=y_test, y_pred=radforest_pred)))

- The accuracy of random forest model on training set is 1.0
- The accuracy of random forest model on test set is 0.9549972889933128
- Confusion matrix: 
              precision    recall  f1-score   support

           0       0.95      0.97      0.96      5878
           1       0.96      0.94      0.95      5188

    accuracy                           0.95     11066
   macro avg       0.96      0.95      0.95     11066
weighted avg       0.96      0.95      0.95     11066



### <strike>Mô hình Naive Bayes</strike> <font color=red>Tràn RAM ???</font>

In [ ]:
#Bayes_model.fit(X_train.toarray(), y_train)
#print("- The accuracy of Naive Bayes model on training set is {}".format(Bayes_model.score(X_train, y_train)))
#bayes_pred = Bayes_model.predict(X_test)
#print("- The accuracy of logistic model on test set is {}".format(accuracy_score(y_true=y_test, y_pred=bayes_pred)))
#print("- Confusion matrix: \n{}".format(classification_report(y_true=y_test, y_pred=bayes_pred)))

### <strike>Mô hình Support Vector Machine (SVM)</strike>

In [ ]:
# SVM_model.fit(X_train, y_train)
# print("- The accuracy of SVM model on training set is {}".format(SVM_model.score(X_train, y_train)))
# svm_pred = SVM_model.predict(X_test)
# print("- The accuracy of logistic model on test set is {}".format(accuracy_score(y_true=y_test, y_pred=svm_pred)))
# print("- Confusion matrix: \n{}".format(classification_report(y_true=y_test, y_pred=svm_pred)))

## 5. Đánh giá mô hình trên tập dữ liệu mới

Tóm tắt quá trình crawl dữ liệu:
- Sử dụng một extension của Google Chrome đó là [Web Scraper browser extension](https://webscraper.io/).
- *Quá trình crawl dữ liệu được ghi lại trong video, xem tại [đây](https://drive.google.com/drive/folders/1mL-WJZk-nrmRvAIp0Ms2kQdSxLhd77jo?usp=sharing)*


### Load dữ liệu

In [ ]:
NEW_TEST_DATA = pd.read_csv('dataset/test_data.csv')

Tóm tắt thông tin của tập dữ liệu mới

In [ ]:
NEW_TEST_DATA.head()

,headline,is_sarcastic
0,BirthdayForBreonna Marks What Would've Been Br...,0
1,TheShowMustBePaused Was Eclipsed By #BlackOutT...,0
2,"Again, Again, Again!' Exclaims Clapping, Grinn...",1
3,"And Then There Were 23,' Says Wayne Messam Cro...",1
4,And Then Those 12 People Send It To 12 People ...,1


In [ ]:
NEW_TEST_DATA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2159 entries, 0 to 2158
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   headline      2159 non-null   object
 1   is_sarcastic  2159 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 33.9+ KB


Tỉ lệ các lớp trong tập dữ liệu

In [ ]:
NEW_TEST_DATA['is_sarcastic'].value_counts()

0    1156
1    1003
Name: is_sarcastic, dtype: int64

*Nhận xét*: Tỉ lệ xấp xỉ 5/5 nên tập dữ liệu không mất tính cân bằng.

In [ ]:
new_X_test = NEW_TEST_DATA['headline']
new_Y_test = NEW_TEST_DATA['is_sarcastic']

### Rút trích đặc trưng từ tập dữ liệu mới bằng phương từ bộ điển được xây dựng dựa trên tập dữ liệu huấn luyện

**Không quan tâm đến các từ chưa *nhìn thấy* có trong tập dữ liệu mới** 

In [ ]:
new_X_test = cv.transform(new_X_test)

In [ ]:
new_logistic_pred = Logistic_model.predict(new_X_test)
new_kNN_pred = KNN_model.predict(new_X_test)
new_radforest_pred = RandomForest_model.predict(new_X_test)

In [ ]:
# Đánh giá mô hình logistic regression trên tập dữ liệu kiểm thử mới 
print("- The accuracy of logistic model on test set is {}".format(accuracy_score(y_true=new_Y_test, y_pred=new_logistic_pred)))
print("- Confusion matrix: \n{}".format(classification_report(y_true=new_Y_test, y_pred=new_logistic_pred)))

- The accuracy of logistic model on test set is 0.762389995368226
- Confusion matrix: 
              precision    recall  f1-score   support

           0       0.74      0.85      0.79      1156
           1       0.80      0.66      0.72      1003

    accuracy                           0.76      2159
   macro avg       0.77      0.76      0.76      2159
weighted avg       0.77      0.76      0.76      2159



In [ ]:
# Đánh giá mô hình kNN trên tập dữ liệu kiểm thử mới 
print("- The accuracy of kNN model on test set is {}".format(accuracy_score(y_true=new_Y_test, y_pred=new_kNN_pred)))
print("- Confusion matrix: \n{}".format(classification_report(y_true=new_Y_test, y_pred=new_kNN_pred)))

- The accuracy of kNN model on test set is 0.6104678091709125
- Confusion matrix: 
              precision    recall  f1-score   support

           0       0.60      0.84      0.70      1156
           1       0.65      0.35      0.45      1003

    accuracy                           0.61      2159
   macro avg       0.62      0.59      0.57      2159
weighted avg       0.62      0.61      0.58      2159



In [ ]:
# Đánh giá mô hình Random Forest trên tập dữ liệu kiểm thử mới 
print("- The accuracy of Random Forest model on test set is {}".format(accuracy_score(y_true=new_Y_test, y_pred=new_radforest_pred)))
print("- Confusion matrix: \n{}".format(classification_report(y_true=new_Y_test, y_pred=new_radforest_pred)))

- The accuracy of Random Forest model on test set is 0.7744326076887448
- Confusion matrix: 
              precision    recall  f1-score   support

           0       0.75      0.86      0.80      1156
           1       0.81      0.67      0.74      1003

    accuracy                           0.77      2159
   macro avg       0.78      0.77      0.77      2159
weighted avg       0.78      0.77      0.77      2159



**Nhận xét**

Mô hình cho kết quả không tốt trên tập dữ liệu kiểm thử so với kết quả của tập dữ liệu kiểm thử được phân chia từ tập dữ liệu ban đầu có sẵn.  <i>Tập dữ liệu mới có chứa các từ `unseen` vì vậy tập từ điển mà chúng ta xây dựng trong khâu rút trích đặc trưng không bao quát được dẫn đến các classifier của chúng ta không thể `capture` được thông tin từ các từ mới này.</i>

---


*Tham khảo*

- [How to Develop a Deep Learning Bag-of-Words Model for Sentiment Analysis (Text Classification)](https://machinelearningmastery.com/deep-learning-bag-of-words-model-sentiment-analysis/)
- [sklearn.feature_extraction.text.CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)
- [Web Scaper Browser Extension](https://webscraper.io/)
- [Removing stop words with NLTK in Python](https://www.geeksforgeeks.org/removing-stop-words-nltk-python/#:~:text=Stop%20Words%3A%20A%20stop%20word,result%20of%20a%20search%20query.&text=To%20check%20the%20list%20of,commands%20in%20the%20python%20shell.)

<b><i><center>---- Hết ----</center></i></b>